In [1]:
import os

In [2]:
%pwd

'c:\\Users\\DELL\\Desktop\\endtoend-mlops\\projects\\covid_chest_classifier\\notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\DELL\\Desktop\\endtoend-mlops\\projects\\covid_chest_classifier'

In [ ]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Yusuf-Abol/endtoend-mlops-MLflow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Yusuf-Abol"
os.environ["MLFLOW_TRACKING_PASSWORD"]="1ed48983c97e28f11ce699668993977aeca9bdb3"

In [ ]:
import dagshub
dagshub.init(repo_owner='Yusuf-Abol', repo_name='endtoend-mlops', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)